In [ ]:
import os
import hashlib
import shutil
import sys
from typing import Dict, List, Optional, Generator, Any
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter  
import numpy as np
from insightface.app import FaceAnalysis
from PIL import Image, ExifTags

# --- HEIC/HEIF Support ---
# This is required to make PIL.Image.open() support HEIC/HEIF formats.
# You must install this library: pip install pillow-heif
try:
    import pillow_heif
    pillow_heif.register_heif_opener()
    print("HEIC/HEIF support enabled.")
except ImportError:
    print("Warning: 'pillow-heif' not installed. HEIC/HEIF files will be skipped.")
    print("Install with: pip install pillow-heif")
# --- End HEIC/HEIF Support ---

try:
    from insightface.app import FaceAnalysis
    print("FaceAnalysis support enabled.")
except ImportError:
    print("Warning: FaceAnalysis not installed.")
    print("pip install insightface")
# --- End HEIC/HEIF Support ---

# --- Hugging Face transformers ---
# You must install this library: pip install transformers torch
try:
    from transformers import pipeline, Pipeline
    from transformers import DetrImageProcessor, DetrForObjectDetection
    import torch
except ImportError:
    print("CRITICAL: 'transformers' or 'torch' not found.")
    print("Please install them to run this script: pip install transformers torch")
    sys.exit(1)


class ImageDataloader:
    """
    Scans a directory for unique images and provides batches for processing.

    This class is implemented as a Python generator. It does not
    inherit from torch.utils.data.DataLoader, as our use case
    requires a simple, stateful iterator.
    """
    IMAGE_EXTENSIONS: tuple = ('.jpg', '.jpeg', '.png', '.heic', '.heif')

    def __init__(self, root_dir: str, batch_size: int = 32):
        if not os.path.isdir(root_dir):
            raise ValueError(f"Root directory not found: {root_dir}")
        if batch_size <= 0:
            raise ValueError("Batch size must be greater than 0")

        self.root_dir = root_dir
        self.batch_size = batch_size
        
        # self.labels will hold the "working state" of all images
        # {image_path: "unknown"}
        self.labels: Dict[str, str] = {}
        self._scan_and_deduplicate()

    def _calculate_hash(self, filepath: str, block_size: int = 65536) -> str:
        """
        Calculates the SHA256 hash of a file's content.
        """
        sha256 = hashlib.sha256()
        try:
            with open(filepath, 'rb') as f:
                while chunk := f.read(block_size):
                    sha256.update(chunk)
            return sha256.hexdigest()
        except (IOError, OSError) as e:
            print(f"Warning: Could not read file for hashing: {filepath}. Skipping. Error: {e}")
            return ""

    def _scan_and_deduplicate(self):
        """
        Walks the root directory, finds all unique images, and
        populates self.labels with the default 'unknown' label.
        """
        print(f"Scanning directory: {self.root_dir}...")
        image_hashes: set[str] = set()
        total_files = 0
        duplicates_skipped = 0

        for root, _, files in os.walk(self.root_dir):
            for file in files:
                if not file.lower().endswith(self.IMAGE_EXTENSIONS):
                    continue

                total_files += 1
                full_path = os.path.join(root, file)
                file_hash = self._calculate_hash(full_path)

                if not file_hash:
                    continue

                if file_hash not in image_hashes:
                    image_hashes.add(file_hash)
                    # All images start as 'unknown'
                    self.labels[full_path] = "unknown"
                else:
                    duplicates_skipped += 1

        print("--- Scan Complete ---")
        print(f"Total image files found: {total_files}")
        print(f"Duplicate images skipped: {duplicates_skipped}")
        print(f"Total unique images to process: {len(self.labels)}")
        if not self.labels:
            print("Warning: No valid, unique images were found.")

    def __len__(self) -> int:
        """
        Returns the total number of unique images to be processed.
        """
        return len(self.labels)

    def __iter__(self) -> Generator[Dict[str, str], None, None]:
        """
        Yields batches of images as dictionaries {image_path: label}.
        """
        # Get a static list of paths to iterate over
        all_paths = list(self.labels.keys())
        
        for i in range(0, len(all_paths), self.batch_size):
            batch_paths = all_paths[i : i + self.batch_size]
            
            # Create the batch dict
            batch_data = {path: self.labels[path] for path in batch_paths}
            
            if batch_data:
                yield batch_data


class ImageClassifier:
    """
    Uses a "waterfall" method to classify images from a dataloader
    using multiple, chained AI models.
    """
    def __init__(self, dataloader: ImageDataloader, output_dir: str = "output"):
        self.dataloader = dataloader
        self.output_dir = output_dir
        
        # this variable save final prediction
        self.processed_images = {}

        # 1. Determine the device
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.pipeline_device = 0 if self.device == torch.device('cuda') else -1
        print(f"Using device: {self.device} (Pipeline device: {self.pipeline_device})")
        
        self.output_paths = {
            "people": os.path.join(output_dir, "people"),
            "view": os.path.join(output_dir, "view"),
            "screen_shot": os.path.join(output_dir, "screen_shot"),
            "unknown": os.path.join(output_dir, "unknown")
        }
        self._create_output_dirs()
        
        # --- Load Models in __init__ ---
        print("Loading models...")
        
        # Stage 1: Face Detector 
        self.retinaface_model = self.load_retinaface_model()
        self.detr_pipeline = self.load_detr_pipeline()

        # Stage 2: View Detector 
        self.clip_pipeline = self.load_clip_pipeline()

        print("Model loading complete.")


    def _create_output_dirs(self):
        """Creates the output directories if they don't exist."""
        print(f"Ensuring output directories exist at: {self.output_dir}")
        for path in self.output_paths.values():
            os.makedirs(path, exist_ok=True)


    def _move_file(self, batch_data: Dict[str, str], action='move'):
        """
        batch_data: dict {image_path: label}

        Moves a batch of image files to their classified directories.
        """
        assert action in ['move', 'copy']
        # Iterate over each image path and its assigned label in the dictionary
        for image_path, new_label in batch_data.items():
            try:
                # 1. Get destination directory. Default to 'unknown' if label is invalid.
                dest_dir = self.output_paths.get(new_label, self.output_paths["unknown"])
                
                # 2. Get the base filename from the full path
                filename = os.path.basename(image_path)
                
                # 3. Create the full destination path
                dest_path = os.path.join(dest_dir, filename)
                

                # 5. Move the file
                if action=='move':
                    shutil.move(image_path, dest_path)
                else:
                    shutil.copy2(image_path, dest_path)


            except FileNotFoundError:
                print(f"File not found, cannot move: {image_path}")
            except PermissionError:
                print(f"Permission denied, cannot move: {image_path}")
            except Exception as e:
                print(f"Failed to move {image_path}: {e}")


    def _display_batch(self, batch_data: Dict[str, str], show_image=False):
        """
        Displays a summary of batch data (category counts) and optionally
        renders each image with its category as the title.

        Args:
            batch_data (Dict[str, str]): Dictionary where keys are file paths 
                                        and values are the categories/labels.
            show_image (bool): A flag to control whether to display images.
        """
        # --- Batch Summary Section ---
        
        # Use collections.Counter to efficiently count the occurrences of each 
        # unique value (category) in the batch_data dictionary.
        value_counts = Counter(batch_data.values())
        
        print("Counts for each unique value in this batch:")
        
        # Loop through the resulting Counter object (value_counts.items())
        for value, count in value_counts.items():
            # Prints each unique value and its total count
            print(f"  Image category '{value}': {count} time(s)")
            
        print("-----------------------------------------")
        
        # --- Image Display Section ---
        
        # This entire block is conditional. It only runs if show_image=True
        if show_image:
            for key in batch_data:
                # Print the file path being processed
                print(key)
                plt.title(batch_data[key])
                plt.imshow(Image.open(key).convert('RGB'))
                plt.show()


    def classify_images(self):
        """
        The main classification loop (orchestrator).
        Iterates through all batches and applies the waterfall logic.
        """
        print("\n--- Starting Image Classification Waterfall ---")
        total_batches = (len(self.dataloader) + self.dataloader.batch_size - 1) // self.dataloader.batch_size
        
        for i, initial_batch in enumerate(self.dataloader):
            print(f"\nProcessing Batch {i+1} / {total_batches} (Size: {len(initial_batch)})...")
            
            remaining_batch = initial_batch.copy()
            # --- STAGE 0: Detect screen shot ---
            remaining_batch = self.check_screen_shot(remaining_batch)

            # --- STAGE 1: Detect faces ---
            if self.retinaface_model:
                remaining_batch = self.detect_human_detr_pipeline(remaining_batch, self.detr_pipeline)
            
            # --- STAGE 1: Detect people ---
            if self.retinaface_model:
                remaining_batch = self.detect_with_RetinaFace(remaining_batch, self.retinaface_model)

            # --- STAGE 2: View Models ---
            if self.clip_pipeline:
                remaining_batch = self.detect_view_clip_pipeline(
                    remaining_batch, self.clip_pipeline,
                    target_label="view",
                    prompts=["a photo of a landscape", "a beautiful view", "a flower", "a photo of food", "a city skyline", "a beach", "mountains", "a forest"]
                )


            self._move_file(remaining_batch, 'copy')
            self.processed_images |=remaining_batch
            print(f"Batch {i+1} complete.")
        
        self._display_batch(self.processed_images)

        print("\n--- Image Classification Finished ---")

    # --- Specific Model Classification Methods ---
    # --- Specific retinaface Model ---
    def load_retinaface_model(self):
        # try:
        #     app = FaceAnalysis(name="buffalo_l")  # uses RetinaFace + ArcFace
        #     app.prepare(ctx_id=0, det_size=(640, 640))  # GPU: ctx_id=0
        #     print("Loaded: retinaface")
        # except Exception as e:
        #     print(f"CRITICAL: Failed to load retinaface. {e}")
        #     app = None

        app = FaceAnalysis(name="buffalo_l")  # uses RetinaFace + ArcFace
        app.prepare(ctx_id=0, det_size=(640, 640))  # GPU: ctx_id=0
        return app
    
    def detect_with_RetinaFace(self, batch_data, model, target_label='people'):
        """
        batch_data: dict {image_path: label}
        model: RetinaFace model (insightface FaceAnalysis)

        Processes only images labeled 'unknown'.
        Returns updated batch_data with detections.
        """
        updated_data = batch_data.copy()

        # 1️⃣ Filter for unknown images
        unknown_items = [(path, label) for path, label in batch_data.items() if label == "unknown"]
        if not unknown_items:
            return updated_data
        
        #print(f"\nProcessing {len(unknown_items)} 'unknown' images using the RetinaFace model...")

        image_paths = [p for p, _ in unknown_items]

        # 2️⃣ Load all images once
        loaded_images = {}
        for img_path in image_paths:
            try:
                img = Image.open(img_path).convert("RGB")
                img_bgr = np.array(img)[:, :, ::-1]  # RGB → BGR
                loaded_images[img_path] = img_bgr
            except Exception as e:
                print(f"⚠️ Error reading {img_path}: {e}")
                updated_data[img_path] = "invalid"

        update_count=0
        # 3️⃣ Run inference per image (insightface doesn't support batch)
        for img_path, img_bgr in loaded_images.items():
            faces = model.get(img_bgr)  # must be called one by one

            if len(faces) > 0:
                updated_data[img_path] = target_label
                update_count+=1

        print(f"\nSuccessfully processed {len(unknown_items)} images using detr. Updated {update_count} labels to '{target_label}'.")
        return updated_data

    # --- Specific detr Model ---
    def load_detr_pipeline(self):
        try:
            model = "facebook/detr-resnet-50"
            detr_pipeline = pipeline("object-detection", model=model, device=self.pipeline_device)
            print("Loaded: facebook/detr-resnet-50")
        except Exception as e:
            print(f"CRITICAL: Failed to load DETR. {e}")
            detr_pipeline = None
        return detr_pipeline

    def detect_human_detr_pipeline(self, batch_data: Dict[str, str], detr_pipeline: Any, target_label='people') -> Dict[str, str]:
        """
        Detects humans ('person' label) in ALL images labeled 'unknown' using the 
        Hugging Face pipeline in a single batch operation and updates the labels.

        Args:
            batch_data (dict): A dictionary of {image_path: label}.
            detr_pipeline (transformers.Pipeline): The loaded DETR object detection pipeline.
            confidence_threshold (float): Minimum confidence for a detection to be considered.

        Returns:
            dict: The updated batch_data dictionary.
        """
        # DETR is trained on COCO, where the label for a human is 'person'
        PERSON_LABEL = 'person' 
        confidence_threshold = 0.9

        if detr_pipeline is None:
            print("ERROR: Pipeline is not loaded. Cannot process data.")
            return batch_data

        # 1. Identify images to process, load, and validate paths
        unknown_paths = [path for path, label in batch_data.items() if label == 'unknown']
        
        if not unknown_paths:
            print("No images labeled 'unknown' found. Returning original data.")
            return batch_data
        
        # print(f"\nProcessing all {len(unknown_paths)} 'unknown' images using the detr pipeline...")

        # Load PIL Images for the pipeline
        batch_images: List[Image.Image] = []
        valid_paths: List[str] = []
        for path in unknown_paths:
            try:
                if not os.path.exists(path):
                    print(f"⚠️ Warning: Image not found at {path}. Skipping.")
                    continue
                batch_images.append(Image.open(path).convert("RGB"))
                valid_paths.append(path)
            except Exception as e:
                print(f"❌ An error occurred loading {path}: {e}. Skipping.")

        if not valid_paths:
            print("No valid images could be loaded. Returning original data.")
            return batch_data

        # 2. Perform single batch inference
        # The pipeline handles moving data to the device defined during load.
        # We pass the list of PIL images directly.
        try:
            # The result is a list of lists: [[det1, det2, ...], [det1, ...], ...]
            results: List[List[Dict[str, Any]]] = detr_pipeline(batch_images)
        except Exception as e:
            print(f"\nRUNTIME ERROR during pipeline execution: {e}")
            print("The batch might be too large for available memory.")
            return batch_data

        update_count = 0
        # 3. Post-process the results
        for idx, image_results in enumerate(results):
            image_path = valid_paths[idx]
            is_human_detected = False
            
            # image_results is a list of dictionaries (one for each detection)
            for detection in image_results:
                # The pipeline provides the score and the label (e.g., 'person')
                if detection['score'] >= confidence_threshold and detection['label'] == PERSON_LABEL:
                    is_human_detected = True
                    # print(f"✅ Detected {PERSON_LABEL} in: {image_path} with score {detection['score']:.2f}")
                    break # Found a person, no need to check other detections for this image

            if is_human_detected:
                batch_data[image_path] = target_label
                update_count+=1

        print(f"\nSuccessfully processed {len(valid_paths)} images using detr. Updated {update_count} labels to '{target_label}'.")

        return batch_data

    # --- Specific clip Model ---
    def load_clip_pipeline(self):
        """
        Loads the CLIP zero-shot classification pipeline.
        """
        try:
            clip_pipeline = pipeline(
                "zero-shot-image-classification",
                model="openai/clip-vit-large-patch14",
                device=self.pipeline_device  
            )
            print("Loaded: clip-vit-large-patch14")
            return clip_pipeline  # <-- FIX: You must return the loaded pipeline
        except Exception as e:
            print(f"CRITICAL: Failed to load CLIP. {e}")
            return None # Return None on failure

    def detect_view_clip_pipeline(self, batch_data: Dict[str, str], clip_pipeline: Any, target_label: str, prompts: List[str], confidence_threshold: float = 0.80) -> Dict[str, str]:
        """
        Classifies images labeled 'unknown' using the CLIP pipeline and a list of prompts.
        
        If the top-scoring prompt meets the confidence threshold, the image label
        is updated to the single `target_label`.

        Args:
            batch_data (dict): A dictionary of {image_path: label}.
            clip_pipeline (transformers.Pipeline): The loaded CLIP pipeline.
            target_label (str): The new label to assign if a match is found (e.g., 'view').
            prompts (list): A list of candidate labels to check against (e.g., "a flower").
            confidence_threshold (float): Minimum confidence for a classification.

        Returns:
            dict: The updated batch_data dictionary.
        """
        if clip_pipeline is None:
            print("ERROR: CLIP Pipeline is not loaded. Cannot process data.")
            return batch_data

        # 1. Identify images to process, load, and validate paths
        unknown_paths = [path for path, label in batch_data.items() if label == 'unknown']
        
        if not unknown_paths:
            print("No images labeled 'unknown' found for CLIP processing. Returning original data.")
            return batch_data
        
        # print(f"\nProcessing {len(unknown_paths)} 'unknown' images using CLIP with {len(prompts)} prompts...")

        # Load PIL Images for the pipeline
        batch_images: List[Image.Image] = []
        valid_paths: List[str] = []
        for path in unknown_paths:
            try:
                if not os.path.exists(path):
                    print(f"⚠️ Warning: Image not found at {path}. Skipping.")
                    continue
                batch_images.append(Image.open(path).convert("RGB"))
                valid_paths.append(path)
            except Exception as e:
                print(f"❌ An error occurred loading {path}: {e}. Skipping.")

        if not valid_paths:
            print("No valid images could be loaded for CLIP. Returning original data.")
            return batch_data

        # 2. Perform single batch inference
        # We pass the list of PIL images and the candidate labels
        try:
            # The result is a list of lists: [[class1, class2, ...], [class1, ...], ...]
            results: List[List[Dict[str, Any]]] = clip_pipeline(
                batch_images, 
                candidate_labels=prompts
            )
        except Exception as e:
            print(f"\nRUNTIME ERROR during CLIP pipeline execution: {e}")
            print("The batch might be too large for available memory.")
            return batch_data

        # 3. Post-process the results
        update_count = 0
        for idx, image_results in enumerate(results):
            image_path = valid_paths[idx]
            
            # The pipeline returns all prompts, sorted by score.
            # We only care about the top-scoring one.
            top_detection = image_results[0]
            
            # Check if the top score meets our threshold.
            # Since the pipeline only returns labels from our `prompts` list,
            # we know the label is one we are looking for.
            if top_detection['score'] >= confidence_threshold:
                # print(f"✅ Classified {image_path} as '{top_detection['label']}' (score: {top_detection['score']:.2f}). Setting label to '{target_label}'.")
                
                # Update the label to the generic target_label
                batch_data[image_path] = target_label
                update_count += 1
            else:
                # The top score was too low, so we "leave it" (it remains 'unknown')
                pass

        print(f"\nSuccessfully processed {len(valid_paths)} images using CLIP. Updated {update_count} labels to '{target_label}'.")
        return batch_data

    # --- Specific screen shot Model ---
    def check_screen_shot(self, batch_data: Dict[str, str], target_label='screen_shot') -> Dict[str, str]:
        """
        Classifies images labeled 'unknown' using a two-stage heuristic method.
        
        1. Disqualifies images with camera-specific EXIF data.
        2. Qualifies images that lack camera EXIF and match known iPhone screen resolutions.
        
        Updated the image label in batch_data to `target_label` or 'photo'.

        Args:
            batch_data (dict): A dictionary of {image_path: label}.
            target_label (str): The new label to assign if a match is found (e.g., 'screen_shot').

        Returns:
            dict: The updated batch_data dictionary.
        """
        
        # --- Start of self-contained logic ---
        
        # 1. Define necessary constants and lookups
        
        # We need to get the string names for the numeric EXIF tag IDs
        # This creates a lookup like {271: 'Make', 272: 'Model', ...}
        try:
            TAGS_LOOKUP = {v: k for k, v in ExifTags.TAGS.items()}
        except AttributeError:
            # Fallback in case ExifTags.TAGS isn't available (though it should be)
            TAGS_LOOKUP = {}

        # EXIF tags that strongly indicate a photo was taken by a camera
        CAMERA_EXIF_TAGS = {
            "Make", "Model", "ExposureTime", "FNumber", 
            "ISOSpeedRatings", "DateTimeOriginal", "LensModel", "GPSInfo"
        }
        
        # A list of (width, height) tuples for known iPhone physical resolutions
        _IPHONE_RESOLUTIONS_BASE = [
            (750, 1334),   # iPhone 6 / 6S / 7 / 8 / SE (2nd/3rd gen)
            (1080, 1920),  # iPhone 6+ / 6S+ / 7+ / 8+ (actual rendered resolution)
            (640, 1136),   # iPhone SE (1st gen) / 5S
            (1125, 2436),  # iPhone X / XS / 11 Pro (5.8″)
            (828, 1792),   # iPhone XR / 11 (6.1″)
            (1242, 2688),  # iPhone XS Max / 11 Pro Max (6.5″)
            (1080, 2340),  # iPhone 12 mini / 13 mini (5.4″)
            (1170, 2532),  # iPhone 12 / 12 Pro / 13 / 13 Pro / (14 6.1″) / (15 6.1″) — verify model variant
            (1284, 2778),  # iPhone 12 Pro Max / 13 Pro Max / (14 Plus) / (15 Plus) — verify model variant
            (1179, 2556),  # iPhone 14 Pro (6.1″) / iPhone 15 Pro (6.1″) — spec confirms 2556×1179. :contentReference[oaicite:18]{index=18}
            (1206, 2622),  # iPhone 16 Pro (6.3″) / iPhone 17 (6.3″) — confirms 2622×1206. :contentReference[oaicite:19]{index=19}
            (1320, 2868),  # iPhone 16 Pro Max / iPhone 17 Pro Max (6.9″) — spec confirm 2868×1320. :contentReference[oaicite:20]{index=20}
        ]
        
        # Create the final set we'll check against
        # Add both (W, H) and (H, W) to catch portrait/landscape
        IPHONE_SCREEN_SIZES = set()
        for w, h in _IPHONE_RESOLUTIONS_BASE:
            IPHONE_SCREEN_SIZES.add((w, h))
            IPHONE_SCREEN_SIZES.add((h, w))
            
        # --- End of self-contained logic ---
        
        update_count = 0
        # Iterate over a static list of keys to safely modify the dictionary
        for image_path in list(batch_data.keys()):
            label = batch_data[image_path]
            
            # Only process images labeled 'unknown'
            if label != 'unknown':
                continue
                
            try:
                with Image.open(image_path) as img:
                    
                    # --- Heuristic 1: Check for DISQUALIFYING camera EXIF data ---
                    # We default to assuming it's not a photo until proven.
                    is_photo = False
                    
                    # Suppress PIL.Image.DecompressionBombWarning if images are very large
                    # This check is basic; for production, you'd set Image.MAX_IMAGE_PIXELS
                    
                    exif_data = img.getexif()
                    if exif_data:
                        for tag_id, value in exif_data.items():
                            # Look up the tag name (e.g., 271 -> 'Make')
                            tag_name = TAGS_LOOKUP.get(tag_id, tag_id)
                            
                            if tag_name in CAMERA_EXIF_TAGS:
                                is_photo = True
                                break
                    
                    if is_photo:
                        # This is almost certainly a camera photo.
                        # Re-label it and stop processing this image.
                        batch_data[image_path] = 'photo'
                        continue 

                    # --- Heuristic 2: Check for QUALIFYING screen dimensions ---
                    # This code only runs if the image had NO EXIF data, 
                    # or had EXIF data but no camera-specific tags.
                    dimensions = img.size  # (width, height)
                    
                    if dimensions in IPHONE_SCREEN_SIZES:
                        # Matched a known resolution and wasn't a photo
                        batch_data[image_path] = target_label
                        update_count+=0
                    # else:
                        # It remains 'unknown' if it doesn't match
            

            except Image.DecompressionBombError:
                print(f"Warning: Image {image_path} is too large to process safely (DecompressionBomb). Labeling as 'error'.")
                batch_data[image_path] = 'processing_error'
            except FileNotFoundError:
                print(f"Error: Image file not found at {image_path}. Labeling as 'error'.")
                batch_data[image_path] = 'file_not_found'
            except Exception as e:
                print(f"Warning: Could not process image {image_path}. Error: {e}")
                batch_data[image_path] = 'processing_error'
        
        print(f"\nSuccessfully processed {len(batch_data)} images using CLIP. Updated {update_count} labels to '{target_label}'.")

        return batch_data


    def place_hold(self, batch_data: Dict[str, str], target_label='screen_shot') -> Dict[str, str]:
        """
        Classifies images labeled 'unknown' using the proposed method.
        
        Updated the image label in batch_data to `target_label`.

        Args:
            batch_data (dict): A dictionary of {image_path: label}.
            target_label (str): The new label to assign if a match is found (e.g., 'screen_shot').

        Returns:
            dict: The updated batch_data dictionary.
        """

c:\Users\xiaom\.conda\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HEIC/HEIF support enabled.
FaceAnalysis support enabled.


In [ ]:

#test_dir = r"D:\images\HockingHills"
test_dir = r"D:\images\images"
output = r"D:\images\output"


# 1. Instantiate the Dataloader
# Using a small batch size for the example
dataloader = ImageDataloader(root_dir=test_dir, batch_size=32)

classifier = ImageClassifier(dataloader=dataloader, output_dir=output)

try:
    classifier.classify_images()
except Exception as e:
    print(f"\nAn error occurred during classification: {e}")
    print("This can happen if you are offline or models are unavailable.")

# 5. Print a summary
print("\n--- Final Output Summary ---")
try:
    for category in os.listdir(output):
        category_path = os.path.join(output, category)
        if os.path.isdir(category_path):
            files = os.listdir(category_path)
            print(f"Files in '{category}': {len(files)} {files}")
except FileNotFoundError:
    print("Output directory 'classification_output' not found. Did the script run?")